# Interpolating z-positions for plate-based imaging
__Keith Cheveralls__<br>
__November 2019__

This notebook contains scripts to facilitate the measurement and interpolation of FocusDrive positions for plate-based imaging. It operates on a list of positions generated by MicroManager's 'HCS Site Generator' plugin and exported from MicroManager's stage-position list/UI.

Its purpose is to compensate for the fact that 96-well imaging plates are tilted with respect to the focal plane of the objective and also, to some extent, not actually planar at all. 

In [ ]:
import os
import re
import sys
import json
import numpy as np
import py4j.protocol

from scipy import interpolate
from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import Axes3D as ax3

sys.path.insert(0, '..')
from dragonfly_automation import operations, utils
from dragonfly_automation.gateway import gateway_utils

gate, mms, mmc = gateway_utils.get_gate(env='prod', wrap=False)

# %load_ext autoreload
# %autoreload 2

### Set the path to the exported position list

In [ ]:
position_list_filepath = 'D://PR//dragonfly-automation-tests//20191106_raw_positions.pos'
with open(position_list_filepath, 'r') as file:
    position_list = json.load(file)

### Measure the FocusDrive position at a subset of wells

Here, we measure the FocusDrive (z-stage) position *after* AFC has been called at each well of a grid-like subset of wells. Detailed instructions for doing so appear below. First, in the cell below, we define the region of the plate to be imaged, and the subset of wells to visit, below. For 'normal' half-plate pipeline imaging, these parameters should not be changed.

In [ ]:
# define the region of the plate to be imaged 
# by specifying the top left and bottom right wells 
# (for half-plate imaging, these should be B2 and G9)
top_left_well_id = 'B2'
bottom_right_well_id = 'G9'

# the list of wells at which to measure FocusDrive positions
# (note that the order here corresponds to the order in which they will be visited below)
well_ids = [
    'B9', 'B5', 'B2',
    'E2', 
    'G2', 'G5', 'G9',
    'E9', 'E5', 
    'B9',
]

# initialize a dict, keyed by well_id, of the measured FocusDrive positions
measured_focusdrive_positions = {}

# find the index of the first position in a given well
def well_id_to_position_ind(well_id):
    for ind, p in enumerate(position_list['POSITIONS']):
        if p['LABEL'].startswith(well_id):
            break
    return ind

In [ ]:
# only run this after interpolation is complete to verify that the the AFC is in range
well_ids = [
    'B7', 'B3','D2', 
    'G3', 'G6', 'G9',
    'F9', 'D9', 'D5', 'E5',
    'B9',
]

In [ ]:
# generate the list of well_ids to visit and consume (via .pop())
well_ids_to_visit = well_ids[::-1]

### Visiting each well and obtaining the FocusDrive position

To visit each well in the `well_ids` list, first run the cell immediately below to move the stage to that well. Then, manually check on the microscope screen whether AFC is in range or not. If it is out of range, move the stage up (or, rarely, down) until it is back in range. Once its in range, proceed to the next cell.

If, during this process, you forget what well you're in, please refer to the cell a few cells below about finding the well closest to the current stage position.

In [ ]:
# go to the next well in the well_id list
well_id = well_ids_to_visit.pop()
ind = well_id_to_position_ind(well_id)
print('Going to well %s' % well_id)

try:
    operations.go_to_position(mms, mmc, ind)
except py4j.protocol.Py4JJavaError:
    operations.go_to_position(mms, mmc, ind)    
print('Arrived at well %s' % well_id)

After running the cell above and verifying that the AFC is in range, run the cell below to 'call' AFC and measure the FocusDrive position after AFC has been applied. This position is then automatically stored in the `measured_focusdrive_positions` dictionary. 

In [ ]:
# call AFC (if it is in-range) and insert the updated FocusDrive position
# in the list of measured focusdrive positions
pos_before = mmc.getPosition('FocusDrive')
mmc.fullFocus()
pos_after = mmc.getPosition('FocusDrive')

measured_focusdrive_positions[well_id] = pos_after
print('FocusDrive position before AFC: %s' % pos_before)
print('FocusDrive position after AFC: %s' % pos_after)

Once the cell just above has been run, go back up one more cell and run it again to move the stage to the next well, then repeat the process of manually checking that AFC is in range beflore calling the cell just above once again.

In [ ]:
# run this cell at any time to see the saved FocusDrive positions
measured_focusdrive_positions

### Convenience method to find the well closest to the current stage position

Run these two cells at any time to determine the well closest to the current stage position. This is useful if you forget what well you're in/on.

In [ ]:
# current xy stage position
current_pos = mmc.getXPosition('XYStage'), mmc.getYPosition('XYStage')
current_pos

In [ ]:
# find the well closest the current position
dists = []
for ind, p in enumerate(position_list['POSITIONS']):
    xystage = [d for d in p['DEVICES'] if d['DEVICE'] == 'XYStage'][0]
    dist = np.sqrt(((np.array(current_pos) - np.array([xystage['X'], xystage['Y']]))**2).sum())
    dists.append(dist)
    
ind = np.argmin(dists)
well_id, site_num = utils.parse_hcs_site_label(position_list['POSITIONS'][ind]['LABEL'])
print('Nearest position is in well %s (ind = %d and distance = %d)' % (well_id, ind, min(dists)))

### Visualize the measured positions and the interpolation

Once all of the wells in the `well_ids` list have been visited, and FocusDrive positions recorded in `measured_focusdrive_positions`, run these cells to visualize the positions and the resulting interpolation. This is meant to be a visual sanity check that the positions 'look' reasonable and are roughly planar.

In [ ]:
measured_positions = np.array([
    (*utils.well_id_to_position(well_id), zpos) 
        for well_id, zpos in measured_focusdrive_positions.items()])

interpolator = interpolate.interp2d(
    measured_positions[:, 0], 
    measured_positions[:, 1], 
    measured_positions[:, 2], 
    kind='linear')

topl_x, topl_y = utils.well_id_to_position(top_left_well_id)
botr_x, botr_y = utils.well_id_to_position(bottom_right_well_id)

x = np.linspace(topl_x, botr_x, 50)
y = np.linspace(topl_y, botr_y, 50)
X, Y = np.meshgrid(x, y)
Z = interpolator(x, y)

In [ ]:
fig = plt.figure()
ax = plt.axes(projection='3d')

ax.plot_surface(
    X, Y, Z, rstride=1, cstride=1,
    cmap='viridis', edgecolor='none')

ax.scatter3D(
    measured_positions[:, 0], measured_positions[:, 1], measured_positions[:, 2], color='red')

### Generate the interpolated position list

Finally, run the cell below to generate the new position list with interpolated FocusDrive positions. This new list is saved in the same directory as the original position list. 

In [ ]:
new_position_list_filepath, new_position_list = utils.interpolate_focusdrive_positions_from_all(
    position_list_filepath,
    measured_focusdrive_positions,
    top_left_well_id,
    bottom_right_well_id)

print('Interpolated position list saved to %s' % new_position_list_filepath)

### Visualize the interpolated positions

In [ ]:
def xyz_from_pos(pos):

    well_id, site_num = utils.parse_hcs_site_label(pos['LABEL'])
    focusdrive = [d for d in pos['DEVICES'] if d['DEVICE']=='FocusDrive'][0]
    x, y = utils.well_id_to_position(well_id)
    z = focusdrive['X']
    return x, y, z

pos = np.array([xyz_from_pos(p) for p in new_position_list['POSITIONS']])
pos.shape

In [ ]:
fig = plt.figure()
ax = plt.axes(projection='3d')

ax.scatter3D(pos[:, 0], pos[:, 1], pos[:, 2], color='gray')

ax.scatter3D(
    measured_positions[:, 0], 
    measured_positions[:, 1], 
    measured_positions[:, 2], 
    color='red')

In [ ]:
fig = plt.figure()
ax = plt.subplot()

ax.scatter(
    pos[:, 0], 
    pos[:, 1], 
    np.abs(pos[:, 2] - 7500) + 10, 
    color='gray')

ax.scatter(
    measured_positions[:, 0], 
    measured_positions[:, 1], 
    np.abs(measured_positions[:, 2] - 7500) + 10, 
    color='red')

### Crop an existing position list

Specify a list of well_ids and a number of sites per well to which to crop the list of positions. This is intended for manual redos in which the user selects, by hand, a few FOVs in a small number of wells for which no acceptable FOVs were imaged during the automated acquisition.

In [ ]:
# test filepath
position_list_filepath = '/Users/keith.cheveralls/image-data/dragonfly-automation-tests/HCS_sites_20191009_INTERPOLATED.pos'

In [ ]:
position_list_filepath = 'D://PR//dragonfly-automation-tests//'

In [ ]:
with open(position_list_filepath, 'r') as file:
    position_list = json.load(file)

In [ ]:
well_ids = ['B9', 'G9', 'A1']
num_sites_per_well = 4
site_nums = range(num_sites_per_well)

positions = position_list['POSITIONS']
positions = [p for p in positions if p['LABEL'].split('-')[0] in well_ids and int(p['LABEL'].split('_')[-1]) in site_nums]

In [ ]:
# check for well_ids that were not in the position_list
missing_well_ids = set(well_ids).difference([p['LABEL'].split('-')[0] for p in positions])
print('Warning: well_ids %s were not found in the position list' % missing_well_ids)

In [ ]:
# view the position labels explicitly
[p['LABEL'] for p in positions]

In [ ]:
# save the cropped position list
cropped_position_list = position_list.copy()
cropped_position_list['POSITIONS'] = positions

dst_filepath = position_list_filepath.replace('.pos', '_CROPPED.pos')
with open(dst_filepath, 'w') as file:
    json.dump(cropped_position_list, file)